In [2]:
import torch
import torch.nn as nn

class FF(nn.Module):
    def __init__(self, embd_dim):
        super().__init__()
        self.linear1 = nn.Linear(embd_dim, 8 * embd_dim)
        self.linear2 = nn.Linear(8 * embd_dim, embd_dim)
        self.gelu = nn.GELU()

    def forward(self, x):
        return self.linear2(self.gelu(self.linear1(x)))

In [57]:
class Head(nn.Module):
    def __init__(self, n_embd, head_size, seq_length):
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(n_embd, self.head_size, bias=False)
        self.query = nn.Linear(n_embd, self.head_size, bias=False)
        self.values = nn.Linear(n_embd, self.head_size, bias=False)
        self.scale_factor = self.head_size**-0.5
        self.seq_length = seq_length

    def forward(self, q, k, v):
        k = self.key(k)
        q = self.query(q)
        v = self.values(v)

        w = (q @ k.transpose(-2, -1)) * self.scale_factor

        mask = torch.tril(torch.ones(self.seq_length, self.seq_length, device=w.device)).unsqueeze(0)
        w = w.masked_fill(mask == 0, float('-inf'))  
        w = nn.functional.softmax(w, dim=-1)
        return w @ v

In [58]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, embd_dim, seq_length):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                Head(embd_dim, embd_dim // num_heads, seq_length)
                for i in range(num_heads)
            ]
        )
        self.out = nn.Linear(embd_dim, embd_dim)

    def forward(self, q, k, v):
        head_out = [head(q, k, v) for head in self.heads]
        concat = torch.cat(head_out, dim=-1)
        return self.out(concat)

In [59]:
class Decode(nn.Module):
    def __init__(self, num_heads, embd_dim, seq_length):
        super().__init__()
        self.attn1 = MultiHeadAttention(num_heads, embd_dim, seq_length)
        self.attn2 = MultiHeadAttention(num_heads, embd_dim, seq_length)
        self.norm1 = nn.LayerNorm(embd_dim)
        self.norm2 = nn.LayerNorm(embd_dim)
        self.norm3 = nn.LayerNorm(embd_dim)
        self.ff = FF(embd_dim)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.2)

    def forward(self, x):
        x_norm = self.norm1(x)
        attn_out = self.attn1(x_norm, x_norm, x_norm)
        x = x + self.dropout1(attn_out)
        x_norm = self.norm2(x)  
        attn_out = self.attn2(x_norm, x_norm, x_norm)
        x = x + self.dropout2(attn_out)
        x_norm = self.norm3(x)  
        x = x + self.dropout3(self.ff(x_norm))  
        return x


In [60]:
class Decoder(nn.Module):
    def __init__(
        self,
        vocab_size,
        seq_length,
        num_layers,
        num_heads,
        embd_dim,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embd_dim)
        self.pos_embedding = nn.Embedding(seq_length, embd_dim)
        self.layers = nn.ModuleList(
            [Decode(num_heads, embd_dim, seq_length) for i in range(num_layers)]
        )
        self.norm = nn.LayerNorm(embd_dim)

    def forward(self, x):
        seq_length = x.size(1)
        positions = (
            torch.arange(0, seq_length, device=x.device).unsqueeze(0).expand_as(x)
        )
        x1 = self.embedding(x) + self.pos_embedding(positions)
        for layer in self.layers:
            x1 = layer(x1)
        return self.norm(x1)

In [ ]:
class GPT3(nn.Module):
    def __init__(self, vocab_size, seq_length, num_heads, num_layers, embd_dim):
        super().__init__()
        self.dec = Decoder(vocab_size, seq_length, num_heads, num_layers, embd_dim)
        self.out = nn.Linear(embd_dim, vocab_size)
        self.seq_length = seq_length
        self.vocab_size = vocab_size

    def forward(self, x):
        x = self.dec(x)
        x = self.out(x)
        return x

    def generate(self, input_ids, max_length=50, temperature=0.9):
        self.eval()
        output = input_ids.tolist()[0]
        with torch.no_grad():
            for _ in range(max_length):
                input_ids = input_ids.to("cuda")
                logits = self(input_ids)
                logits = logits[:, -1, :] / temperature
                probs = nn.functional.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                output.append(int(next_token[0, 0]))
                input_ids = torch.cat([input_ids[:, 1:], next_token], dim=1)
        self.train()
        return output

In [ ]:
vocab_size = 50000 # dont touch this
seq_length = 128 # length of the input sequence, gpt3 uses 1024, use a smaller value for gpu constraints
num_heads = 12 # number of attention heads, gpt3 uses 96, use a smaller value such that embd dim is divisible by num_heads
num_layers = 12 # number of transformer blocks, gpt3 uses 96, use a smaller value 
embd_dim = 768 # embedding dimension, gpt3 uses 12288, use a smaller value
model = GPT3(vocab_size, seq_length, num_heads, num_layers, embd_dim)

In [ ]:
input_ids = torch.randint(0, vocab_size, (1, seq_length))
model(input_ids).shape
model.generate(input_ids)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 246,975,824
